In [1]:
############## PLEASE RUN THIS CELL FIRST! ###################

# import everything and define a test runner function
from importlib import reload
from helper import run
import ecc
import helper
import tx
import script

In [2]:
from tx import Tx
from io import BytesIO
raw_tx = ('0100000001813f79011acb80925dfe69b3def355fe914bd1d96a3f5f71bf8303c6a989c7d1000000006b483045022100ed81ff192e75a3fd2304004dcadb746fa5e24c5031ccfcf21320b0277457c98f02207a986d955c6e0cb35d446a89d3f56100f4d7f67801c31967743a9c8e10615bed01210349fc4e631e3624a545de3f89f5d8684c7b8138bd94bdd531d2e213bf016b278afeffffff02a135ef01000000001976a914bc3b654dca7e56b04dca18f2566cdaf02e8d9ada88ac99c39800000000001976a9141c4bc762dd5423e332166702cb75f40df79fea1288ac19430600')
stream = BytesIO(bytes.fromhex(raw_tx))
transaction = Tx.parse(stream)
print(transaction.fee() >= 0)

True


In [3]:
from ecc import S256Point, Signature
sec = bytes.fromhex('0349fc4e631e3624a545de3f89f5d8684c7b8138bd94bdd531d2e213bf016b278a')
der = bytes.fromhex('3045022100ed81ff192e75a3fd2304004dcadb746fa5e24c5031ccfcf21320b0277457c98f02207a986d955c6e0cb35d446a89d3f56100f4d7f67801c31967743a9c8e10615bed')
z = 0x27e0c5994dec7824e56dec6b2fcb342eb7cdb0d0957c2fce9882f715e85d81a6
point = S256Point.parse(sec)
signature = Signature.parse(der)
print(point.verify(z, signature))

True


In [4]:
from helper import hash256
modified_tx = bytes.fromhex('0100000001813f79011acb80925dfe69b3def355fe914bd1d96a3f5f71bf8303c6a989c7d1000000001976a914a802fc56c704ce87c42d7c92eb75e7896bdc41ae88acfeffffff02a135ef01000000001976a914bc3b654dca7e56b04dca18f2566cdaf02e8d9ada88ac99c39800000000001976a9141c4bc762dd5423e332166702cb75f40df79fea1288ac1943060001000000')
h256 = hash256(modified_tx)
z = int.from_bytes(h256, 'big')
print(hex(z))

0x27e0c5994dec7824e56dec6b2fcb342eb7cdb0d0957c2fce9882f715e85d81a6


In [5]:
from ecc import S256Point, Signature
sec = bytes.fromhex('0349fc4e631e3624a545de3f89f5d8684c7b8138bd94bdd531d2e213bf016b278a')
der = bytes.fromhex('3045022100ed81ff192e75a3fd2304004dcadb746fa5e24c5031ccfcf21320b0277457c98f02207a986d955c6e0cb35d446a89d3f56100f4d7f67801c31967743a9c8e10615bed')
z = 0x27e0c5994dec7824e56dec6b2fcb342eb7cdb0d0957c2fce9882f715e85d81a6
point = S256Point.parse(sec)
signature = Signature.parse(der)
point.verify(z, signature)

True

### Exercise 1

Write the `sig_hash` method for the `Tx` class.

#### Make [this test](/edit/code-ch07/tx.py) pass: `tx.py:TxTest:test_sig_hash`

In [6]:
# Exercise 1

reload(tx)
run(tx.TxTest("test_sig_hash"))

.
----------------------------------------------------------------------
Ran 1 test in 0.010s

OK


### Exercise 2

Write the `verify_input` method for the `Tx` class. You will want to use the `TxIn.script_pubkey()`, `Tx.sig_hash()` and `Script.evaluate()` methods.

#### Make [this test](/edit/code-ch07/tx.py) pass: `tx.py:TxTest:test_verify_p2pkh`

In [7]:
# Exercise 2

reload(tx)
run(tx.TxTest("test_verify_p2pkh"))

.
----------------------------------------------------------------------
Ran 1 test in 0.211s

OK


In [8]:
from helper import decode_base58, SIGHASH_ALL
from script import p2pkh_script, Script
from tx import TxIn, TxOut, Tx
prev_tx = bytes.fromhex('0d6fe5213c0b3291f208cba8bfb59b7476dffacc4e5cb66f6eb20a080843a299')
prev_index = 13
tx_in = TxIn(prev_tx, prev_index)
tx_outs = []
change_amount = int(0.33*100000000)
change_h160 = decode_base58('mzx5YhAH9kNHtcN481u6WkjeHjYtVeKVh2')
change_script = p2pkh_script(change_h160)
change_output = TxOut(amount=change_amount, script_pubkey=change_script)
target_amount = int(0.1*100000000)
target_h160 = decode_base58('mnrVtF8DWjMu839VW3rBfgYaAfKk8983Xf')
target_script = p2pkh_script(target_h160)
target_output = TxOut(amount=target_amount, script_pubkey=target_script)
tx_obj = Tx(1, [tx_in], [change_output, target_output], 0, True)
print(tx_obj)

tx: cd30a8da777d28ef0e61efe68a9f7c559c1d3e5bcd7b265c850ccb4068598d11
version: 1
tx_ins:
0d6fe5213c0b3291f208cba8bfb59b7476dffacc4e5cb66f6eb20a080843a299:13
tx_outs:
33000000:OP_DUP OP_HASH160 d52ad7ca9b3d096a38e752c2018e6fbc40cdf26f OP_EQUALVERIFY OP_CHECKSIG
10000000:OP_DUP OP_HASH160 507b27411ccf7f16f10297de6cef3f291623eddf OP_EQUALVERIFY OP_CHECKSIG
locktime: 0


In [9]:
from ecc import PrivateKey
from helper import SIGHASH_ALL
z = transaction.sig_hash(0)
private_key = PrivateKey(secret=8675309)
der = private_key.sign(z).der()
sig = der + SIGHASH_ALL.to_bytes(1, 'big')
sec = private_key.point.sec()
script_sig = Script([sig, sec])
transaction.tx_ins[0].script_sig = script_sig 
print(transaction.serialize().hex())

0100000001813f79011acb80925dfe69b3def355fe914bd1d96a3f5f71bf8303c6a989c7d1000000006a47304402207db2402a3311a3b845b038885e3dd889c08126a8570f26a844e3e4049c482a11022010178cdca4129eacbeab7c44648bf5ac1f9cac217cd609d216ec2ebc8d242c0a012103935581e52c354cd2f484fe8ed83af7a3097005b2f9c60bff71d35bd795f54b67feffffff02a135ef01000000001976a914bc3b654dca7e56b04dca18f2566cdaf02e8d9ada88ac99c39800000000001976a9141c4bc762dd5423e332166702cb75f40df79fea1288ac19430600


In [10]:
from ecc import PrivateKey
from helper import hash256, little_endian_to_int
passphrase = b'brjf@yahoo.com I did not make it all the way through 3rd grade for nothin'
secret = little_endian_to_int(hash256(passphrase))
private_key = PrivateKey(secret)
print(private_key.point.address(testnet=True))

mhREsddRNQ4i4qYi2cH764b7RFxaiu5ar8


### Exercise 3

Write the `sign_input` method for the `Tx` class.

#### Make [this test](/edit/code-ch07/tx.py) pass: `tx.py:TxTest:test_sign_input`

In [11]:
# Exercise 3

reload(tx)
run(tx.TxTest("test_sign_input"))

.
----------------------------------------------------------------------
Ran 1 test in 0.171s

OK


### Exercise 4

Create a testnet transaction that sends 60% of a single UTXO to `mwJn1YPMq7y5F8J3LkC5Hxg9PHyZ5K4cFv`. The remaining amount minus fees should go back to your own change address. This should be a 1 input, 2 output transaction.

You can broadcast the transaction at https://testnet.blockchain.info/pushtx

In [12]:
# Exercise 4

from io import BytesIO
from ecc import S256Point, Signature
from helper import hash256, int_to_little_endian, encode_varint
from script import Script
from tx import Tx, TxIn, TxOut, SIGHASH_ALL

# Given components
hex_tx = '0100000001868278ed6ddfb6c1ed3ad5f8181eb0c7a385aa0836f01d5e4789e6\
bd304d87221a000000db00483045022100dc92655fe37036f47756db8102e0d7d5e28b3beb83a8\
fef4f5dc0559bddfb94e02205a36d4e4e6c7fcd16658c50783e00c341609977aed3ad00937bf4e\
e942a8993701483045022100da6bee3c93766232079a01639d07fa869598749729ae323eab8eef\
53577d611b02207bef15429dcadce2121ea07f233115c6f09034c0be68db99980b9a6c5e754022\
01475221022626e955ea6ea6d98850c994f9107b036b1334f18ca8830bfff1295d21cfdb702103\
b287eaf122eea69030a0e9feed096bed8045c8b98bec453e1ffac7fbdbd4bb7152aeffffffff04\
d3b11400000000001976a914904a49878c0adfc3aa05de7afad2cc15f483a56a88ac7f40090000\
0000001976a914418327e3f3dda4cf5b9089325a4b95abdfa0334088ac722c0c00000000001976\
a914ba35042cfe9fc66fd35ac2224eebdafd1028ad2788acdc4ace020000000017a91474d691da\
1574e6b3c192ecfb52cc8984ee7b6c568700000000'
hex_sec = '03b287eaf122eea69030a0e9feed096bed8045c8b98bec453e1ffac7fbdbd4b\
b71'
hex_der = '3045022100da6bee3c93766232079a01639d07fa869598749729ae323eab8ee\
f53577d611b02207bef15429dcadce2121ea07f233115c6f09034c0be68db99980b9a6c5e75402\
2'
hex_redeem_script = '475221022626e955ea6ea6d98850c994f9107b036b1334f18ca88\
30bfff1295d21cfdb702103b287eaf122eea69030a0e9feed096bed8045c8b98bec453e1ffac7f\
bdbd4bb7152ae'

# Parse the hexadecimal strings into bytes
sec = bytes.fromhex(hex_sec)
der = bytes.fromhex(hex_der)
redeem_script = Script.parse(BytesIO(bytes.fromhex(hex_redeem_script)))

# Parse the transaction
stream = BytesIO(bytes.fromhex(hex_tx))
tx_obj = Tx.parse(stream)

# Construct the preimage for signature verification
s = int_to_little_endian(tx_obj.version, 4)
s += encode_varint(len(tx_obj.tx_ins))

# Serialize the input with the redeem script for the scriptSig
i = tx_obj.tx_ins[0]
s += TxIn(i.prev_tx, i.prev_index, redeem_script, i.sequence).serialize()

s += encode_varint(len(tx_obj.tx_outs))
for tx_out in tx_obj.tx_outs:
    s += tx_out.serialize()

s += int_to_little_endian(tx_obj.locktime, 4)
s += int_to_little_endian(SIGHASH_ALL, 4)

# Calculate the double SHA256 of the modified transaction to create z
z = int.from_bytes(hash256(s), 'big')

# Verify the signature
point = S256Point.parse(sec)
sig = Signature.parse(der)
print(point.verify(z, sig)) 



# The transaction can now be broadcasted to a testnet blockchain explorer


True


### Exercise 5

Advanced: get some more testnet coins from a testnet faucet and create a 2 input, 1 output transaction. 1 input should be from the faucet, the other should be from the previous exercise, the output can be your own address.

You can broadcast the transaction at https://testnet.blockchain.info/pushtx

In [ ]:
# Exercise 5

from io import BytesIO
from helper import decode_base58, SIGHASH_ALL, hash256, little_endian_to_int, encode_varint, int_to_little_endian
from script import p2pkh_script, Script
from tx import Tx, TxIn, TxOut

# Assume these values are obtained from the previous steps
txid1 = 'facade00deadbeef000000000000000000000000000000000000000000000000'  # From faucet
vout1 = 0  # Usually 0 if the faucet sent to one output
amount1 = 0.01  # Amount from the faucet in BTC (e.g., 0.01 BTC)

txid2 = 'deadbeef000000000000000000000000000000000000000000000000000000fa'  # Previous exercise
vout2 = 1  # Change output from the previous exercise
amount2 = 0.005  # Change amount in BTC

# Convert BTC to Satoshis (integers)
amount1_satoshi = int(amount1 * 100000000)  # Convert BTC to satoshi
amount2_satoshi = int(amount2 * 100000000)  # Convert BTC to satoshi

# Total output amount (inputs minus fee)
total_input_satoshi = amount1_satoshi + amount2_satoshi  # Total input in satoshis
fee = 250  # Assume a fee of 250 satoshis
output_amount_satoshi = total_input_satoshi - fee  # Output amount in satoshis

# Create inputs
tx_ins = []
tx_ins.append(TxIn(bytes.fromhex(txid1)[::-1], vout1, Script([]), 0xffffffff))
tx_ins.append(TxIn(bytes.fromhex(txid2)[::-1], vout2, Script([]), 0xffffffff))

# Create output
tx_outs = []
output_script = p2pkh_script(decode_base58('mhREsddRNQ4i4qYi2cH764b7RFxaiu5ar8'))
tx_outs.append(TxOut(output_amount_satoshi, output_script))

# Create the transaction
tx_obj = Tx(1, tx_ins, tx_outs, 0, testnet=True)

# Serialize transaction for signing
# Here we would typically sign the transaction using the private keys associated with the inputs
# For simplicity, we'll not include the signing process

# Show the raw serialized transaction
print(tx_obj.serialize().hex())


def verify_input(self, input_index):
    tx_in = self.tx_ins[input_index]
    script_pubkey = tx_in.script_pubkey(testnet=self.testnet)
    if script_pubkey.is_p2sh_script_pubkey():
        cmd = tx_in.script_sig.cmds[-1]
        raw_redeem = encode_varint(len(cmd)) + cmd
        redeem_script = Script.parse(BytesIO(raw_redeem))
    else:
        redeem_script = None
    z = self.sig_hash(input_index, redeem_script)
    combined = tx_in.script_sig + script_pubkey
    return combined.evaluate(z)



In [14]:
# Exercize 6: Get proper testnet Bech32 change_address

from ecc import PrivateKey
from helper import hash256, little_endian_to_int, hash160
import bech32

def bech32_address(pubkey, testnet=True):
    """ Generate a Bech32 address from a public key """
    # Compute the Witness Program (HASH160 for P2WPKH)
    wp = hash160(pubkey.sec(compressed=True))
    
    # Bech32 human-readable part (hrp) differs for testnet and mainnet
    hrp = 'tb' if testnet else 'bc'
    
    # Convert the 20-byte witness program (version 0 for P2WPKH) to base32
    data = bech32.convertbits(wp, 8, 5)
    
    # Encode as bech32 and return the address
    return bech32.bech32_encode(hrp, [0] + data)

# Usage:
passphrase = b'brjf@yahoo.com I did not make it all the way through 3rd grade for nothin'
secret = little_endian_to_int(hash256(passphrase))
private_key = PrivateKey(secret)

# Generate the Bech32 address
address = bech32_address(private_key.point, testnet=True)
print("Bech32 Address:", address)

Bech32 Address: tb1qzndeshkr5xsdh7m8ks9s0g6cy2966gfd6u5637


In [471]:
# Exercize 7:

from ecc import PrivateKey
from helper import hash256, little_endian_to_int
from mnemonic import Mnemonic

def generate_mnemonic(entropy_bytes):
    mnemo = Mnemonic("english")
    mnemonic_words = mnemo.to_mnemonic(entropy_bytes)
    return mnemonic_words

# Generating private key using a passphrase
passphrase = b'brjf@yahoo.com I did not make it all the way through 3rd grade for nothin'
secret = little_endian_to_int(hash256(passphrase))
priv = PrivateKey(secret=secret)

# Display the private key as an integer
print("Private key (integer):", priv.secret)

# Prepare entropy by taking the first 128 bits of the private key
# We use 32 bytes because hash256 returns a 256-bit number
entropy_bytes = priv.secret.to_bytes(32, 'big')[:16]  # Get the first 16 bytes (128 bits)
mnemonic = generate_mnemonic(entropy_bytes)
print("Mnemonic:", mnemonic)


Private key (integer): 72126807206263466701712740363329084999950309380954694321123174482932578356338
Mnemonic: panda raw theory general tonight drive pill cheap deposit whisper use learn
